In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F


import warnings
warnings.filterwarnings("ignore")

# Loading Dataset

In [ ]:
train = pd.read_csv("../input/Kannada-MNIST/train.csv")
test = pd.read_csv("../input/Kannada-MNIST/test.csv")
train.head(3)

In [ ]:
test.head(3)

# Feature and Label data

In [ ]:
X = train.drop('label', axis=1)
test = test
y = train.label

# Using PyTorch Linear Layer

In [ ]:
X = train.iloc[:, 1:].values
y = train.iloc[:, 0].values
print("Shape of X is :{} Shape of y is :{}".format(X.shape, y.shape))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=.1, random_state=42, stratify=y)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


In [ ]:
class Model(nn.Module):
    def __init__(self, input_features=784, hidden_layer1=392, hidden_layer2=196, output_features=10):
        super().__init__()
        self.fc1 = nn.Linear(input_features,hidden_layer1)                  
        self.fc2 = nn.Linear(hidden_layer1, hidden_layer2)                  
        self.out = nn.Linear(hidden_layer2, output_features)      
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [ ]:
Model = Model()

In [ ]:
#optimizer = torch.optim.SGD(Model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(Model.parameters(), lr=0.001)
#loss = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 100
train_losses = []
valid_losses = []
train_correct = []
valid_correct = []

for epoch in range(epochs):
    out1 = Model.forward(x_train)
    train_loss = criterion(out1, y_train)
    train_losses.append(train_loss)
    
    out2 = Model.forward(x_test)
    valid_loss = criterion(out2, y_test)
    valid_losses.append(valid_loss)

    optimizer.zero_grad()
    train_loss.backward()
    valid_loss.backward()
    optimizer.step()
    
    print(f'Epoch: {epoch:2}  Train_loss: {train_loss.item():10.8f}  Valid_loss:{valid_loss.item():10.8f} ')


In [ ]:
plt.plot(range(epochs), train_losses)
plt.ylabel("Loss")
plt.xlabel("epoch")

plt.plot(range(epochs), valid_losses)
plt.ylabel("Loss")
plt.xlabel("epoch")

In [ ]:
predicted = Model(x_test)
_, predict_y = torch.max(predicted, 1)

In [ ]:
print('prediction accuracy :', accuracy_score(y_test.data, predict_y.data))

In [ ]:
# Classification Report
print(classification_report(y_test.data, predict_y.data))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, predict_y.data)
plt.figure(figsize=(7,7))
sns.heatmap(cm, fmt='.0f', annot=True, linewidths=0.2, linecolor='purple')
plt.xlabel('predicted value')
plt.ylabel('Truth value')
plt.show()

In [ ]:
sub = pd.read_csv("../input/Kannada-MNIST/sample_submission.csv") 
test = pd.read_csv("../input/Kannada-MNIST/test.csv")
sub.head(3)

In [ ]:
test_df = test.drop('id', axis=1).values
test_df = torch.FloatTensor(test_df)
    
predicted = Model(test_df)
_, predict_test = torch.max(predicted, 1)
predicted_data = predict_test.data
predicted_data

In [ ]:
predicted_data

In [ ]:
sub['label'] = predicted_data
sub.to_csv('predicted.csv', index=False) 
sub.head()